In [8]:
import json
import requests
import bs4 ,requests ,time ,os ,argparse ,sys
import pandas as pd 

## for request_api function
from datetime import datetime


In [10]:
url = 'https://disco.deliveryhero.io/search/api/v1/feed'
payload = {
    'q': 'pizza',
    'location': {
        'point': {
            'longitude': 96.1719225,  # 經度
            'latitude': 16.7727507  # 緯度
        }
    },
    'config': 'Original',
    'vertical_types': ['restaurants'],
    'include_component_types': ['vendors'],
    'include_fields': ['feed'],
    'language_id': '3',
    'opening_type': 'delivery',
    'platform': 'web',
    'language_code': 'my',
    'customer_type': 'regular',
    'limit': 100,  # no.of restaurants
    'offset': 0,  # Offset value, use when you want to get more information
    'dynamic_pricing': 0,
    'brand': 'foodpanda',
    'country_code': 'mm',
    'use_free_delivery_label': False
}
headers = {
    'content-type': "application/json",
}
r = requests.post(url=url, data=json.dumps(payload), headers=headers)
if r.status_code == requests.codes.ok:
    data = r.json()
    restaurants = data['feed']['items'][0]['items']
    print(len(restaurants))
    for restaurant in restaurants[:5]:
        print(restaurant['name'])
else:
    print('Something')

54
PEZZO Pizza Kannar
Royal 9 Pizza & Food House
FUDO bakery ( Pabedan )
FUDO Bakery ( Myanma Gone yi )
Parami Pizza(Downtown)


In [50]:
## Scrape by City 
mainURL = "https://www.foodpanda.com.mm/"
engURL = "https://www.foodpanda.com.mm/en/?r=1"
result = requests.get(mainURL)

## City URLs
html_m = bs4.BeautifulSoup(result.content, "html.parser")
all_a = html_m.find_all("a", class_="city-tile")   # popular find_all can be used without it , read its documentation
all_link = [mainURL + "en" +a.get("href") for a in all_a]

## City Names - Myanmar
all_cityTitle = html_m.find_all("span", class_="city-name")
all_cityTitleName_MM = [c.text.strip() for c in all_cityTitle]

## Use two different links becuase of FP shit, their english page need '?r=1' thing
## City Names - English 
result_e = requests.get(engURL)
html_e = bs4.BeautifulSoup(result_e.content, "html.parser")
all_cityName = html_e.find_all("span", class_="city-name")
cityNames = [cityName.text.strip() for cityName in all_cityName]
# print(cityNames)
# print(all_link)

city_lvl = []
total_city = 0
total_city += len(all_link)
total_store = 0

for cityIndex in range(0,len(all_link)):
    InCity = requests.get(all_link[cityIndex])
    print(u'area : '+ all_cityTitleName_MM[cityIndex])
    
    for title in range(0,len(cityNames)):
        InCity_pg = bs4.BeautifulSoup(InCity.content,"html.parser")
        InCityAllA = InCity_pg.find_all("a", class_="hreview-aggregate url")

        ## Collect store links from each city 
        InCityAllLink = [b.get("href") for b in InCityAllA]

        InCityAllid = [c.get("data-vendor-id") for c in InCityAllA]
        StoreN= InCity_pg.find_all("span", class_="name fn")
        storeNames = [s.text for s in StoreN]

    for i in range(0,len(InCityAllLink)):
        city_lvl.append(
            {
                'City_Name': cityNames[cityIndex],
                'City_Page': all_link[cityIndex],
                'Restaurant_link' : InCityAllLink[i],
                'ID': InCityAllid[i],
                'Restaurant_Name': storeNames[i]
            }
        )
        # for StoreIndex in range(0, len(InCityAllLink)):
        #     StoreResult = requests.get(InCityAllLink[StoreIndex])
        #     Store_pg = bs4.BeautifulSoup(StoreResult.content, "html.parser")
        
    ## Make total number of store
    total_store += len(InCityAllLink)
    print("Total No. of URLs for {} is {}.".format(cityNames[cityIndex],len(InCityAllLink)))

with open('city_lvl.json','w', encoding='utf-8') as outfile:
    json.dump(city_lvl, outfile, indent=4)
    print("Total No. of stores {} in {} cities".format(total_store,total_city))


area : ရန်ကုန်
Total No. of URLs for Yangon is 297.
area : မန္တလေး
Total No. of URLs for Mandalay is 294.
area : ပုဂံ
Total No. of URLs for Bagan is 216.
area : မော်လမြိုင်
Total No. of URLs for Mawlamyine is 300.
area : ပြင်ဦးလွင်
Total No. of URLs for Pyin Oo Lwin is 299.
area : နေပြည်တော်
Total No. of URLs for Naypyitaw is 300.
area : ပဲခူး
Total No. of URLs for Bago is 244.
area : မုံရွာ
Total No. of URLs for Monywa is 94.
area : တောင်ငူ
Total No. of URLs for Taungoo is 102.
area : ပုသိမ်
Total No. of URLs for Pathein is 65.
area : ပခုက္ကူ
Total No. of URLs for Pakokku is 5.
area : တောင်ကြီး
Total No. of URLs for Taung Gyi is 298.
area : မုဒုံ
Total No. of URLs for Mudon is 27.
area : လားရှိုး
Total No. of URLs for Lashio is 68.
Total No. of stores 2609 in 14 cities


In [12]:
pcode = pd.read_csv('pcode.csv')
drop_col = ['FID', 'the_geom', 'Lat_Long' ,'Level', 'Source', 'Remark']
new_df = pcode.drop(drop_col, axis=1).loc[pcode['State_Divi']=='Yangon']
# print(new_df.columns)
# print(new_df.head())

test = new_df.tail(5)
for i,row in test.iterrows():
    print(row['Township'], row['Longitude'], row['Latitude'])
    lat_val = row['Latitude']
    long_val = row['Longitude']
    ts_name = row['Township']
    get_links = 1
    count = 0
    count_final = 0
    
    json_resp = request_api(lat_val, long_val, ts_name)
    json_resp = json.loads(json_resp)
    if json_resp.get("status_code","") == 200:
        for loop_data in json_resp.get("data", {}).get("items",[]):
            print("API start")
            restaurant_listing_data = dict()
            restaurant_listing_data["input_latitude"] = lat_val
            restaurant_listing_data["input_longitude"] = long_val
            restaurant_listing_data["listing_id"] = loop_data.get("id","")
            restaurant_listing_data["listing_city"] = loop_data.get("city",{})
            restaurant_listing_data["listing_cuisines"] = loop_data.get("cuisines","")
            restaurant_listing_data["listing_food_characteristics"] = loop_data.get("food_characteristics","")
            restaurant_listing_data["listing_latitude"] = loop_data.get("latitude","")
            restaurant_listing_data["listing_longitude"] = loop_data.get("longitude","")
            # restaurant_listing_data["listing_minimum_delivery_fee"] = loop_data.get("minimum_delivery_fee","")
            # restaurant_listing_data["listing_minimum_delivery_time"] = loop_data.get("minimum_delivery_time","")
            # restaurant_listing_data["listing_minimum_order_amount"] = loop_data.get("minimum_order_amount","")
            # restaurant_listing_data["listing_minimum_pickup_time"] = loop_data.get("minimum_pickup_time","")
            restaurant_listing_data["listing_name"] = loop_data.get("name","")
            restaurant_listing_data["listing_post_code"] = loop_data.get("post_code","")
            restaurant_listing_data["listing_rating"] = loop_data.get("rating","")
            restaurant_listing_data["listing_review_number"] = loop_data.get("review_number","")
            restaurant_listing_data["listing_url"] = loop_data.get("redirection_url","")
            restaurant_listing_data["listing_score"] = loop_data.get("score","")
            # restaurant_listing_data["listing_service_fee_percentage_amount"] = loop_data.get("service_fee_percentage_amount","")
            # restaurant_listing_data["listing_service_tax_percentage_amount"] = loop_data.get("service_tax_percentage_amount","")
            # restaurant_listing_data["listing_special_days"] = loop_data.get("special_days",[])
            restaurant_listing_data["listing_web_path"] = loop_data.get("web_path","")
            restaurant_listing_data["listing_website"] = loop_data.get("website","")
            restaurant_listing_data["listing_has_online_payment"] = loop_data.get("has_online_payment","")
            restaurant_listing_data["listing_discounts_info"] = loop_data.get("discounts","")
            url_crawled = loop_data.get("redirection_url","")
            res_zipcode = loop_data.get("post_code","")
            for k,v in loop_data.items():
                if k.startswith("is_"):
                    restaurant_listing_data["listing_"+k] = v
            file_write = open('foodpanda.json', 'a')
            file_write.write(json.dumps(restaurant_listing_data)+"\n")
            file_write.close()
            print("API Done")
            
        try:
            print("Try start!")
            count_final = json_resp.get("data",{}).get("availalbe_count","")
            count = count + json_resp.get("data",{}).get("returned_count","")
            if count >= count_final:
                get_links = 0
                print("Data {} rows are fetched for {}".format(count,ts_name))
            else:
                #get_links = 1
                print("Need offset?")
        except Exception as e:
            count_final = json_resp.get("data",{}).get("availalbe_count","")
            count = count + json_resp.get("data",{}).get("returned_count","")
            print("Count values : {} & {}".format(count_final,count))
            # print("Some error while API")
    else:
        # get_links = 0 
        print("Error , error , Error!!!")

    print("Done for township : {}".format(ts_name))
print("Everything done!")


Kamaryut 96.1329 16.8236
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API start
API Done
API st

NameError: name 'count' is not defined

In [7]:
def request_api(lat_val,long_val,json_name):
    url = "https://disco.deliveryhero.io/listing/api/v1/pandora/vendors?latitude="+str(lat_val)+"&longitude="+str(long_val)+"&language_id=3&include=characteristics&Connection=keep-alive&dynamic_pricing=0&configuration=Variant&country=mm&vertical=restaurants&customer_type=regular&limit=&offset=&cuisine&budgets&customer_hash"
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
      'Accept': 'application/json, text/plain, */*',
      'Accept-Language': 'en-US,en;q=0.5',
      'X-FP-API-KEY': 'volo',
      'dps-session-id': 'eyJzZXNzaW9uX2lkIjoiMjEyMGJkZTU4ZGYyMTgxZWMxZmFkOGQwM2RjN2ZmOWEiLCJwZXJzZXVzX2lkIjoiMTYyODkyMDI5My40ODU1NDE4MTM2LlFWcnFaenZvVEQiLCJ0aW1lc3RhbXAiOjE2MzAyMzc5MjF9',
      'x-disco-client-id': 'web',
      'Origin': 'https://www.foodpanda.com.mm/',
      'Referer': 'https://www.foodpanda.com.mm/',
      'Connection': 'keep-alive',
      'TE': 'Trailers'
    }
    response = requests.request("GET", url, headers=headers)
    now=datetime.now()
    dt_string = now.strftime(r"%d-%m-%Y")
    fs = open(str(json_name) + '.json','w', encoding='utf-8')
    fs.write(response.content.decode("utf-8"))
    fs.close()
    return response.content.decode("utf-8")